In [1]:
# POC for ny taxi

In [1]:
import pandas as pd
import glob

In [2]:
data_file_location = '../resources/data/*.parquet'

In [19]:
# trip data
all_trip_data_df = None
for source_file_name in glob.glob(data_file_location):
    if all_trip_data_df is None:
        all_trip_data_df = pd.read_parquet(source_file_name)
    else:
        pd.concat([all_trip_data_df, pd.read_parquet(source_file_name)], ignore_index=True)

In [20]:
all_trip_data_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_borough,pickup_zone,drop_off_borough,drop_off_zone,year,month
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,Manhattan,Manhattan Valley,Manhattan,Upper West Side South,2019,2019-01
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,Manhattan,Upper West Side South,Manhattan,West Chelsea/Hudson Yards,2019,2019-01
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,Manhattan,Upper East Side North,Manhattan,Upper East Side North,2018,2018-12
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2018,2018-11
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2018,2018-11
...,...,...,...,...,...,...,...,...,...
7667787,2019-01-31 23:57:36,2019-02-01 00:18:39,1,Manhattan,Yorkville West,Manhattan,Alphabet City,2019,2019-02
7667788,2019-01-31 23:32:03,2019-01-31 23:33:11,1,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2019,2019-01
7667789,2019-01-31 23:36:36,2019-01-31 23:36:40,1,Unknown,NV,Unknown,NV,2019,2019-01
7667790,2019-01-31 23:14:53,2019-01-31 23:15:20,1,Unknown,NV,Queens,Astoria,2019,2019-01


In [104]:
all_trip_data_df.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'pickup_borough', 'pickup_zone', 'drop_off_borough', 'drop_off_zone',
       'year', 'month'],
      dtype='object')

In [94]:
sum_up= all_trip_data_df.groupby(['pickup_zone','drop_off_zone']).agg({'passenger_count': ['sum']})
sum_up

passenger_count
                                                            sum
pickup_zone             drop_off_zone                          
Allerton/Pelham Gardens Allerton/Pelham Gardens              22
                        Bayside                               1
                        Bedford                               1
                        Bedford Park                          2
                        Borough Park                          1
...                                                         ...
Yorkville West          Woodlawn/Wakefield                   30
                        Woodside                            184
                        World Trade Center                  251
                        Yorkville East                    15389
                        Yorkville West                    17493

[31821 rows x 1 columns]

In [95]:
sum_up.reset_index()

,pickup_zone,drop_off_zone,passenger_count
,,,sum
0,Allerton/Pelham Gardens,Allerton/Pelham Gardens,22
1,Allerton/Pelham Gardens,Bayside,1
2,Allerton/Pelham Gardens,Bedford,1
3,Allerton/Pelham Gardens,Bedford Park,2
4,Allerton/Pelham Gardens,Borough Park,1
...,...,...,...
31816,Yorkville West,Woodlawn/Wakefield,30
31817,Yorkville West,Woodside,184
31818,Yorkville West,World Trade Center,251


In [105]:
sum_up.columns

MultiIndex([('passenger_count', 'sum')],
           )